Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



## Códigos e discussão



## Conclusão



## Playground

